In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [2]:
print(path)

/kaggle/input/cifake-real-and-ai-generated-synthetic-images


In [3]:
import torch
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms

from transformers import ViTImageProcessor

model_name = "google/vit-large-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

image_mean, image_std = processor.image_mean, processor.image_std

normalize = transforms.Normalize(mean=image_mean, std=image_std)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

CIFAKE_train_dataset = datasets.ImageFolder(root=f"{path}/train", transform=transform)
CIFAKE_test_dataset = datasets.ImageFolder(root=f"{path}/test", transform=transform)

val_ratio = 0.2

targets = CIFAKE_train_dataset.targets

train_indices, val_indices = train_test_split(
    list(range(len(targets))),
    test_size=val_ratio,
    stratify=targets,
    random_state=42
)

CIFAKE_train_dataset = Subset(CIFAKE_train_dataset, train_indices)
CIFAKE_val_dataset = Subset(CIFAKE_train_dataset.dataset, val_indices)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def collate_fn(examples):
    pixel_values, labels = zip(*examples)
    pixel_values = torch.stack(pixel_values)
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

In [8]:
label2id = {'FAKE': 0, 'REAL': 1}
id2label = {0: 'FAKE', 1: 'REAL'}

In [9]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels = 2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    output_dir="output-models",
    num_train_epochs=2,
    eval_strategy = "steps",
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    logging_strategy = "steps",
    logging_steps = 400,
    fp16=True,
    weight_decay=0.02,
    save_strategy = "no",
    dataloader_pin_memory=False
)

trainer = Trainer(
    model,
    train_args,
    train_dataset=CIFAKE_train_dataset,
    eval_dataset=CIFAKE_val_dataset,
    data_collator=collate_fn,
    tokenizer=processor,
)

trainer.model.to(device)
trainer.train()

<ipython-input-10-64381e3ccb26>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sk2338 (sk2338-georgetown-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/watchdog_ai")
processor.save_pretrained("/content/drive/MyDrive/watchdog_ai")

['/content/drive/MyDrive/watchdog_ai_v0/preprocessor_config.json']